In [1]:
import os
import json
import requests
import azure.core

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [16]:


resource_name = "Phishingdetection_rs"
workspace_name = "Phishingdetection_ws"
subscriptionID = "f276ab10-9503-4c82-9d49-04de4c8194c0" 
aml_compute_target = "phishd-cluster" 
experiment_name= 'phishing_expirement'

In [19]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

forced_interactive_auth = InteractiveLoginAuthentication(tenant_id="2d3194e3-1654-46bd-bae2-ad37ba11b0ae", force=True)

# Now create Workspace
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US",
                           auth=auth)
    ws.write_config('.azureml')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/2d3194e3-1654-46bd-bae2-ad37ba11b0ae/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.


get_workspace error using subscription_id=776851b3-7a01-4304-92cb-70f54ac4dea8, resource_group_name=Phishingdetection_rs, workspace_name=Phishingdetection_ws


Deploying KeyVault with name phishingkeyvault093fb81c.
Deploying StorageAccount with name phishingstorage5c177449c.
Deployed KeyVault with name phishingkeyvault093fb81c. Took 93.49 seconds.
Deployed StorageAccount with name phishingstorage5c177449c. Took 102.58 seconds.
Deploying AppInsights with name phishinginsights4c3082af.
Deployed AppInsights with name phishinginsights4c3082af. Took 3.28 seconds.
Deploying Workspace with name Phishingdetection_ws.
Deployed Workspace with name Phishingdetection_ws. Took 21.99 seconds.


In [20]:
Workspace.from_config()


Workspace.create(name='Phishingdetection_ws', subscription_id='f276ab10-9503-4c82-9d49-04de4c8194c0', resource_group='Phishingdetection_rs')

In [21]:
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")

creating new compute target : phishd-cluster
InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached now


In [22]:
exp = Experiment(ws,experiment_name)

In [23]:
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")
config=ScriptRunConfig(source_directory="./",script="phishingModel.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

RunId: phishing_expirement_1746967891_c2a6adeb
Web View: https://ml.azure.com/runs/phishing_expirement_1746967891_c2a6adeb?wsid=/subscriptions/f276ab10-9503-4c82-9d49-04de4c8194c0/resourcegroups/Phishingdetection_rs/workspaces/Phishingdetection_ws&tid=2d3194e3-1654-46bd-bae2-ad37ba11b0ae

Streaming azureml-logs/20_image_build_log.txt

======Starting Image Build on Serverless Compute======
The run ID for the image build on serverless compute is imgbldrun_bb6cd9e
Additional logs for the run: https://ml.azure.com/experiments/id/prepare_image/runs/imgbldrun_bb6cd9e?wsid=/subscriptions/f276ab10-9503-4c82-9d49-04de4c8194c0/resourcegroups/Phishingdetection_rs/workspaces/Phishingdetection_ws&tid=2d3194e3-1654-46bd-bae2-ad37ba11b0ae
2025-05-11T12:56:05: Logging into Docker registry: 020b758c5bc14f6e871330afb0ae3e5f.azurecr.io
2025-05-11T12:56:05: WARNING! Using --password via the CLI is insecure. Use --password-stdin.

2025-05-11T12:56:05: Login Succeeded
2025-05-11T12:56:05: WARNING! Your cred

{'runId': 'phishing_expirement_1746967891_c2a6adeb',
 'target': 'phishd-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-05-11T13:04:57.131745Z',
 'endTimeUtc': '2025-05-11T13:09:04.328401Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  '_azureml.ClusterName': 'phishd-cluster',
  'ContentSnapshotId': '3accd848-9229-4ed8-89b7-6f48bce5857b',
  'azureml.git.repository_uri': 'https://github.com/HassanHAT/PhishingDetectionSystem.git',
  'mlflow.source.git.repoURL': 'https://github.com/HassanHAT/PhishingDetectionSystem.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'da4d195ba02dbbbb1328942bf17ee1714f3b0404',
  'mlflow.source.git.commit': 'da4d195ba02dbbbb1328942bf17ee1714f3b0404',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'p

In [24]:
config=ScriptRunConfig(source_directory="./",script="phishingModel.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7])
execution=exp.submit(config)

In [25]:

#Register model in workspace
from azureml.core import Model
model = Model.register(ws,model_path="./phishingmodel.pkl",model_name="phishingD_ml_model")

Registering model phishingD_ml_model


In [26]:
env.environment_variables


{'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'}

In [27]:
model = Model(ws,"phishingD_ml_model")

In [28]:
myenv=Environment(name="phishingD-env")
conda_packages = ['numpy']
pip_packages = ['azureml-sdk', 'azureml-defaults', 'scikit-learn', 'pandas', 'joblib']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages, python_version='3.8.5')
myenv.python.conda_dependencies=mycondaenv
myenv.register(workspace=ws)

inference_config = InferenceConfig(entry_script='score.py',source_directory='.',environment=myenv)

In [29]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [30]:
# Deploying the model
service = Model.deploy(ws,"phishingdmodel", 
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aciconfig,
                           #deployment_target=predenv,
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

C:\Users\almal\AppData\Local\Temp\ipykernel_23360\491521870.py:2: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws,"phishingdmodel",


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-05-11 16:33:25+03:00 Creating Container Registry if not exists.
2025-05-11 16:33:26+03:00 Building image..
2025-05-11 16:42:39+03:00 Generating deployment configuration.
2025-05-11 16:42:41+03:00 Submitting deployment to compute..
2025-05-11 16:42:47+03:00 Checking the status of deployment phishingdmodel..
2025-05-11 16:44:03+03:00 Checking the status of inference endpoint phishingdmodel.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://8c8f7204-18ea-4817-9651-f2005618615b.eastus.azurecontainer.io/score


In [32]:
# Prepare your test data
import pandas as pd
import json
test_message = "Your account needs verification. Click www.suspicious-link.com"
test_data = json.dumps({"data": [test_message]})  # As list
prediction = service.run(input_data=test_data)
prediction

'{"results": [{"message": "Your account needs verification. Click www.suspicious-link.com", "prediction": "safe", "probability": 18.055592522943524, "risk": {"level": "high", "color": "red"}}]}'